# **Chapter 45: Project 2 - Social Media Feed**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Implement infinite scrolling feeds with cursor-based pagination and loading states
- Handle high-performance media display using `cached_network_image` and video player integration
- Establish real-time WebSocket connections for live feed updates and notifications
- Implement optimistic UI updates for likes and comments with conflict resolution
- Build custom media caching strategies to reduce bandwidth and improve load times
- Structure complex user interactions (threaded comments, content reporting) with clean state management
- Handle sensitive content filtering and user block lists at the data layer

---

## **Prerequisites**

- Completed Chapter 44: Project 1 - E-Commerce App (mastery of Clean Architecture and BLoC)
- Completed Chapter 21: GraphQL & WebSockets (understanding of socket communication)
- Proficiency with `cached_network_image` and video player packages
- Understanding of "Optimistic UI" concepts (updating UI before server confirmation)
- Familiarity with JSON parsing and serialization for complex nested data

---

## **45.1 Architecture Overview**

Unlike the e-commerce app which was CRUD-heavy, a social feed requires **real-time data synchronization** and **optimistic interactions**. We will adapt Clean Architecture to support streaming data sources.

### **Feature Structure**

```
lib/features/feed/
├── data/
│   ├── datasources/
│   │   ├── feed_remote_datasource.dart     # REST API for initial load
│   │   ├── feed_socket_datasource.dart     # WebSocket for real-time
│   │   └── feed_local_datasource.dart      # Cache for viewed posts
│   └── repositories/
│       └── feed_repository_impl.dart
├── domain/
│   ├── entities/
│   │   ├── post.dart
│   │   └── comment.dart
│   ├── repositories/
│   │   └── feed_repository.dart
│   └── usecases/
│       ├── get_feed.dart
│       ├── like_post.dart
│       └── add_comment.dart
└── presentation/
    ├── bloc/
    │   ├── feed_bloc.dart
    │   └── post_interaction_bloc.dart  # Handles likes/comments locally
    ├── pages/
    │   └── feed_page.dart
    └── widgets/
        ├── post_card.dart
        └── comment_section.dart
```

**Explanation:**

- **Dual Data Sources**: `RemoteDataSource` fetches initial feed via REST (reliable, cachable). `SocketDataSource` listens for real-time updates (new posts, like counts) via WebSocket.
- **Interaction Bloc**: Separates "local interactions" (optimistic likes) from "feed loading" (pagination). This prevents the entire feed reloading when a user likes a post.

---

## **45.2 Infinite Scroll with Cursor-Based Pagination**

Cursor-based pagination is superior to offset-based for real-time feeds where new posts are constantly added.

### **API Design**

```json
// GET /api/feed?limit=20&cursor=base64_encoded_timestamp_id
{
  "data": [
    { "id": "p10", "content": "...", "createdAt": "2026-02-13T10:00:00Z" },
    ...
  ],
  "pagination": {
    "next_cursor": "eyJ0cyI6IjIwMjYtMDItMTNUMDk6MDA6MDBaIiwiaWQiOiJwMSJ9",
    "has_more": true
  }
}
```

**Explanation:**

- **Cursor**: Encodes the timestamp and ID of the last fetched item. The server returns items *older* than this cursor.
- **Stability**: Unlike `offset=20`, inserting a new post at the top doesn't shift the window, preventing duplicate items.

### **Data Layer Implementation**

```dart
// lib/features/feed/data/datasources/feed_remote_datasource.dart
@RestApi(baseUrl: "/api")
abstract class FeedRemoteDataSource {
  factory FeedRemoteDataSource(Dio dio) = _FeedRemoteDataSource;

  @GET("/feed")
  Future<FeedResponseModel> getFeed({
    @Query("limit") required int limit,
    @Query("cursor") String? cursor,
  });
}

// lib/features/feed/data/models/feed_response_model.dart
@JsonSerializable()
class FeedResponseModel {
  final List<PostModel> data;
  final PaginationMetaModel pagination;

  FeedResponseModel({required this.data, required this.pagination});

  factory FeedResponseModel.fromJson(Map<String, dynamic> json) =>
      _$FeedResponseModelFromJson(json);
}

@JsonSerializable()
class PaginationMetaModel {
  final String? nextCursor;
  final bool hasMore;

  PaginationMetaModel({this.nextCursor, required this.hasMore});

  factory PaginationMetaModel.fromJson(Map<String, dynamic> json) =>
      _$PaginationMetaModelFromJson(json);
}
```

### **Presentation Layer: Pagination Logic**

```dart
// lib/features/feed/presentation/bloc/feed_bloc.dart
@injectable
class FeedBloc extends Bloc<FeedEvent, FeedState> {
  final GetFeed getFeed;
  final int pageSize = 20;

  FeedBloc(this.getFeed) : super(const FeedState.initial()) {
    on<_FetchFeed>(_onFetchFeed);
    on<_LoadMore>(_onLoadMore);
  }

  Future<void> _onFetchFeed(
    _FetchFeed event,
    Emitter<FeedState> emit,
  ) async {
    emit(const FeedState.loading());

    final result = await getFeed(const FeedParams());

    result.fold(
      (failure) => emit(FeedState.error(failure.message)),
      (response) => emit(FeedState.loaded(
        posts: response.posts,
        hasReachedEnd: !response.hasMore,
        nextCursor: response.nextCursor,
      )),
    );
  }

  Future<void> _onLoadMore(
    _LoadMore event,
    Emitter<FeedState> emit,
  ) async {
    final currentState = state;
    
    // Guard: Don't load if already loading or reached end
    if (currentState is! _Loaded ||
        currentState.isLoadingMore ||
        currentState.hasReachedEnd) {
      return;
    }

    emit(currentState.copyWith(isLoadingMore: true));

    final result = await getFeed(FeedParams(
      cursor: currentState.nextCursor,
    ));

    result.fold(
      (failure) => emit(currentState.copyWith(
        isLoadingMore: false,
        loadError: failure.message,
      )),
      (response) => emit(FeedState.loaded(
        posts: [...currentState.posts, ...response.posts],
        hasReachedEnd: !response.hasMore,
        nextCursor: response.nextCursor,
        isLoadingMore: false,
      )),
    );
  }
}
```

```dart
// lib/features/feed/presentation/pages/feed_page.dart
class FeedPage extends StatefulWidget {
  const FeedPage({super.key});

  @override
  State<FeedPage> createState() => _FeedPageState();
}

class _FeedPageState extends State<FeedPage> {
  final ScrollController _scrollController = ScrollController();

  @override
  void initState() {
    super.initState();
    context.read<FeedBloc>().add(const FeedEvent.fetchFeed());
    
    // Setup scroll listener for infinite scroll
    _scrollController.addListener(_onScroll);
  }

  void _onScroll() {
    if (_isBottom) {
      context.read<FeedBloc>().add(const FeedEvent.loadMore());
    }
  }

  bool get _isBottom {
    if (!_scrollController.hasClients) return false;
    final maxScroll = _scrollController.position.maxScrollExtent;
    final currentScroll = _scrollController.offset;
    // Trigger 200px before actual bottom
    return currentScroll >= (maxScroll * 0.9);
  }

  @override
  void dispose() {
    _scrollController.dispose();
    super.dispose();
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      body: BlocBuilder<FeedBloc, FeedState>(
        builder: (context, state) {
          return state.maybeWhen(
            loading: () => const Center(child: CircularProgressIndicator()),
            error: (msg) => Center(child: Text('Error: $msg')),
            loaded: (posts, hasReachedEnd, _, isLoadingMore, __) {
              if (posts.isEmpty) {
                return const Center(child: Text('No posts yet'));
              }
              
              return RefreshIndicator(
                onRefresh: () async {
                  context.read<FeedBloc>().add(const FeedEvent.fetchFeed());
                },
                child: ListView.builder(
                  controller: _scrollController,
                  physics: const AlwaysScrollableScrollPhysics(),
                  itemCount: posts.length + (hasReachedEnd ? 0 : 1),
                  itemBuilder: (context, index) {
                    // Show loading indicator at the bottom
                    if (index >= posts.length) {
                      return const Center(
                        child: Padding(
                          padding: EdgeInsets.all(16.0),
                          child: CircularProgressIndicator(),
                        ),
                      );
                    }
                    
                    return PostCard(post: posts[index]);
                  },
                ),
              );
            },
            orElse: () => const SizedBox(),
          );
        },
      ),
    );
  }
}
```

**Explanation:**

- **Trigger Threshold**: Triggering load at 90% scroll depth (`maxScroll * 0.9`) ensures a smooth continuation without the user hitting a "hard stop".
- **RefreshIndicator**: Pull-to-refresh emits `FetchFeed` again, resetting the cursor and fetching the latest posts.
- **ListView itemCount**: Adds `+1` to itemCount for the loading indicator at the bottom unless `hasReachedEnd` is true.

---

## **45.3 Media Handling and Caching**

Social feeds are media-heavy. Efficient caching prevents reloading images/videos repeatedly.

### **Image Caching with `cached_network_image`**

```dart
// lib/features/feed/presentation/widgets/post_card.dart
class PostCard extends StatelessWidget {
  final Post post;

  const PostCard({super.key, required this.post});

  @override
  Widget build(BuildContext context) {
    return Card(
      margin: const EdgeInsets.symmetric(vertical: 8, horizontal: 12),
      child: Column(
        crossAxisAlignment: CrossAxisAlignment.start,
        children: [
          // Header
          _PostHeader(author: post.author, timestamp: post.createdAt),
          
          // Content
          Padding(
            padding: const EdgeInsets.all(12.0),
            child: Text(post.content),
          ),
          
          // Media Grid (Handling single or multiple images)
          if (post.media.isNotEmpty) _MediaGrid(media: post.media),
          
          // Actions
          _ActionRow(post: post),
        ],
      ),
    );
  }
}

class _MediaGrid extends StatelessWidget {
  final List<Media> media;

  const _MediaGrid({required this.media});

  @override
  Widget build(BuildContext context) {
    if (media.isEmpty) return const SizedBox();

    // Single image: Full width
    if (media.length == 1) {
      return _CachedMediaItem(media: media.first);
    }

    // Multiple images: Grid layout
    return GridView.builder(
      shrinkWrap: true,
      physics: const NeverScrollableScrollPhysics(),
      gridDelegate: const SliverGridDelegateWithFixedCrossAxisCount(
        crossAxisCount: 3,
        crossAxisSpacing: 2,
        mainAxisSpacing: 2,
      ),
      itemCount: media.length > 4 ? 4 : media.length, // Show max 4
      itemBuilder: (context, index) {
        // Show overlay if more than 4 images
        final isLast = index == 3 && media.length > 4;
        
        return Stack(
          fit: StackFit.expand,
          children: [
            _CachedMediaItem(media: media[index]),
            if (isLast)
              Container(
                color: Colors.black54,
                child: Center(
                  child: Text(
                    '+${media.length - 4}',
                    style: const TextStyle(
                      color: Colors.white,
                      fontSize: 24,
                      fontWeight: FontWeight.bold,
                    ),
                  ),
                ),
              ),
          ],
        );
      },
    );
  }
}

class _CachedMediaItem extends StatelessWidget {
  final Media media;

  const _CachedMediaItem({required this.media});

  @override
  Widget build(BuildContext context) {
    if (media.type == MediaType.video) {
      return _VideoThumbnail(media: media);
    }

    return CachedNetworkImage(
      imageUrl: media.url,
      fit: BoxFit.cover,
      // Thumbnail while loading
      placeholder: (context, url) => Container(
        color: Colors.grey[200],
        child: const Center(
          child: CircularProgressIndicator(strokeWidth: 2),
        ),
      ),
      // Error handling
      errorWidget: (context, url, error) => Container(
        color: Colors.grey[200],
        child: const Icon(Icons.error, color: Colors.grey),
      ),
      // Fade-in animation
      fadeInDuration: const Duration(milliseconds: 300),
      fadeInCurve: Curves.easeIn,
      
      // Cache configuration
      memCacheWidth: 500,  // Downscale for memory efficiency
      maxWidthDiskCache: 1000, // Limit disk cache size
      
      // Progress indicator for large images
      progressIndicatorBuilder: (context, url, downloadProgress) {
        if (downloadProgress.progress == null) return const SizedBox();
        return Center(
          child: CircularProgressIndicator(
            value: downloadProgress.progress,
            strokeWidth: 2,
          ),
        );
      },
    );
  }
}
```

**Explanation:**

- **CachedNetworkImage**: Automatically caches images to disk and memory.
- **memCacheWidth**: Downsamples the image in memory to `500px` width. Critical for preventing OOM (Out of Memory) crashes in infinite lists.
- **maxWidthDiskCache**: Stores the cached image at a maximum resolution. Saves disk space on the device.
- **Placeholder/Error**: Provides visual feedback while loading or if the URL fails.

### **Video Player Integration**

```dart
// lib/features/feed/presentation/widgets/video_player_widget.dart
class _VideoThumbnail extends StatefulWidget {
  final Media media;

  const _VideoThumbnail({required this.media});

  @override
  State<_VideoThumbnail> createState() => _VideoThumbnailState();
}

class _VideoThumbnailState extends State<_VideoThumbnail> {
  late VideoPlayerController _controller;
  bool _isInitialized = false;
  bool _isPlaying = false;

  @override
  void initState() {
    super.initState();
    _initializePlayer();
  }

  Future<void> _initializePlayer() async {
    final url = Uri.parse(widget.media.url);
    
    // Use cached video controller if possible
    _controller = VideoPlayerController.networkUrl(url);
    
    await _controller.initialize();
    
    if (mounted) {
      setState(() => _isInitialized = true);
      _controller.setLooping(true);
    }
  }

  @override
  void dispose() {
    _controller.dispose();
    super.dispose();
  }

  void _togglePlayback() {
    setState(() {
      if (_controller.value.isPlaying) {
        _controller.pause();
        _isPlaying = false;
      } else {
        _controller.play();
        _isPlaying = true;
      }
    });
  }

  @override
  Widget build(BuildContext context) {
    if (!_isInitialized) {
      return Container(
        color: Colors.black,
        child: const Center(child: CircularProgressIndicator()),
      );
    }

    return GestureDetector(
      onTap: _togglePlayback,
      child: Stack(
        alignment: Alignment.center,
        children: [
          AspectRatio(
            aspectRatio: _controller.value.aspectRatio,
            child: VideoPlayer(_controller),
          ),
          
          // Play/Pause Icon Overlay
          if (!_isPlaying)
            Container(
              decoration: const BoxDecoration(
                color: Colors.black45,
                shape: BoxShape.circle,
              ),
              child: const Icon(Icons.play_arrow, size: 48, color: Colors.white),
            ),
            
          // Mute/Volume indicator could go here
        ],
      ),
    );
  }
}
```

**Explanation:**

- **Lifecycle Management**: `VideoPlayerController` must be disposed to free native resources. Failing to dispose causes memory leaks.
- **Initialization Check**: Video loading is async. Only render `VideoPlayer` widget after `_isInitialized` is true to avoid errors.
- **Looping**: `setLooping(true)` is standard for social feeds (like Instagram/TikTok).
- **Performance Note**: In a production app, you would use a singleton or cache manager for video controllers to prevent re-buffering when scrolling back to a video.

---

## **45.4 Real-Time Updates with WebSockets**

Using WebSockets for instant feed updates (new posts, live like counts) without polling.

### **WebSocket Data Source**

```dart
// lib/core/services/web_socket_service.dart
@singleton
class WebSocketService {
  final String socketUrl = 'wss://api.social-app.com/ws';
  WebSocketChannel? _channel;
  final StreamController<Map<String, dynamic>> _streamController = 
      StreamController.broadcast();
  
  Stream<Map<String, dynamic>> get stream => _streamController.stream;
  bool _isConnected = false;

  Future<void> connect() async {
    if (_isConnected) return;

    try {
      _channel = IOWebSocketChannel.connect(Uri.parse(socketUrl));

      _isConnected = true;
      debugPrint('WebSocket connected');

      // Listen to incoming messages
      _channel!.stream.listen(
        (data) {
          final decoded = jsonDecode(data as String) as Map<String, dynamic>;
          _streamController.add(decoded);
        },
        onError: (error) {
          debugPrint('WebSocket error: $error');
          _handleDisconnect();
        },
        onDone: () {
          debugPrint('WebSocket closed by server');
          _handleDisconnect();
        },
      );
    } catch (e) {
      debugPrint('WebSocket connection failed: $e');
      await _reconnect();
    }
  }

  void _handleDisconnect() {
    _isConnected = false;
    _reconnect();
  }

  Future<void> _reconnect() async {
    await Future.delayed(const Duration(seconds: 3));
    await connect();
  }

  void send(Map<String, dynamic> data) {
    if (_isConnected) {
      _channel?.sink.add(jsonEncode(data));
    }
  }

  void disconnect() {
    _channel?.sink.close();
    _isConnected = false;
  }
}
```

**Explanation:**

- **IOWebSocketChannel**: Connects to WebSocket server. `connect()` initiates the handshake.
- **Broadcast Stream**: `_streamController.broadcast()` allows multiple listeners (FeedBloc, ChatBloc) to listen to one socket connection.
- **Reconnection Logic**: Simple exponential backoff (`Future.delayed`) handles temporary network drops.
- **Keep-Alive**: Production apps need `ping/pong` heartbeats to keep connections open through proxies.

### **Integrating WebSocket with BLoC**

```dart
// lib/features/feed/presentation/bloc/feed_bloc.dart additions
  late StreamSubscription _socketSubscription;

  @override
  Future<void> close() {
    _socketSubscription.cancel();
    return super.close();
  }

  void _listenToSocket(WebSocketService socketService) {
    _socketSubscription = socketService.stream.listen((data) {
      final type = data['type'] as String?;
      
      if (type == 'new_post') {
        final post = PostModel.fromJson(data['post']);
        add(FeedEvent.newPostReceived(post));
      } else if (type == 'post_liked') {
        final payload = data['payload'];
        add(FeedEvent.postUpdated(
          postId: payload['post_id'],
          likeCount: payload['like_count'],
        ));
      }
    });
  }

  Future<void> _onNewPostReceived(
    _NewPostReceived event,
    Emitter<FeedState> emit,
  ) async {
    final currentState = state;
    
    if (currentState is _Loaded) {
      // Prepend new post to the list
      emit(currentState.copyWith(
        posts: [event.post, ...currentState.posts],
      ));
    }
  }
```

**Explanation:**

- **Event-Driven Architecture**: WebSocket messages are converted into BLoC events (`FeedEvent.newPostReceived`), keeping the BLoC as the single source of truth.
- **Prepending**: New posts are inserted at the top of the list. The cursor remains valid because the user is viewing older items below.
- **Lifecycle**: Cancel subscription in `close()` to prevent memory leaks when the page is disposed.

---

## **45.5 Optimistic UI Updates**

Optimistic updates make the UI feel instant by updating the local state before waiting for the server response.

### **Like Button Logic**

```dart
// lib/features/feed/presentation/bloc/post_interaction_bloc.dart
@injectable
class PostInteractionBloc extends Bloc<PostInteractionEvent, PostInteractionState> {
  final LikePostRepository likeRepository;

  PostInteractionBloc(this.likeRepository) : super(const PostInteractionState.initial()) {
    on<_LikePost>(_onLikePost);
  }

  Future<void> _onLikePost(
    _LikePost event,
    Emitter<PostInteractionState> emit,
  ) async {
    final currentState = state;
    final postId = event.postId;
    
    // 1. OPTIMISTIC UPDATE: Immediately update UI
    if (currentState is _Loaded) {
      final updatedPosts = currentState.posts.map((post) {
        if (post.id == postId) {
          // Toggle like status
          return post.copyWith(
            isLiked: !post.isLiked,
            likeCount: post.isLiked ? post.likeCount - 1 : post.likeCount + 1,
          );
        }
        return post;
      }).toList();

      emit(PostInteractionState.loaded(posts: updatedPosts));
    }

    // 2. API CALL: Send request to server
    final result = await likeRepository.toggleLike(postId);

    // 3. ROLLBACK (if failed)
    result.fold(
      (failure) {
        // Revert back to original state
        if (currentState is _Loaded) {
          emit(currentState); // Emit previous state
          
          // Show error snackbar
          ScaffoldMessenger.key.currentState?.showSnackBar(
            SnackBar(content: Text('Failed to like post: ${failure.message}')),
          );
        }
      },
      (success) {
        // Success! The optimistic update is now confirmed.
        // No action needed unless the server returns a corrected count.
        if (success.serverLikeCount != null) {
          _correctLikeCount(postId, success.serverLikeCount!);
        }
      },
    );
  }
  
  void _correctLikeCount(String postId, int serverCount) {
    // Optional: Correct the count if server has different number (e.g., someone else liked simultaneously)
    // Implementation similar to optimistic update
  }
}
```

**Explanation:**

- **Immediate Feedback**: The user sees the heart turn red and the count increase immediately.
- **State Duplication**: `post.copyWith()` creates a new immutable state object, triggering the UI rebuild.
- **Conflict Resolution**: If the server disagrees (e.g., due to race conditions), the `_correctLikeCount` method syncs reality.
- **Rollback**: If the API fails (network error), revert to `currentState` and notify the user.

### **UI Implementation of Like Button**

```dart
// lib/features/feed/presentation/widgets/action_row.dart
class _ActionRow extends StatelessWidget {
  final Post post;

  const _ActionRow({required this.post});

  @override
  Widget build(BuildContext context) {
    return Padding(
      padding: const EdgeInsets.symmetric(horizontal: 8.0, vertical: 4.0),
      child: Row(
        children: [
          // Like Button
          BlocBuilder<PostInteractionBloc, PostInteractionState>(
            buildWhen: (previous, current) {
              // Only rebuild if this specific post changed
              // This is a simplified check; in reality, compare post objects
              return true;
            },
            builder: (context, state) {
              final currentPost = state.maybeWhen(
                loaded: (posts) => posts.firstWhere((p) => p.id == post.id, orElse: () => post),
                orElse: () => post,
              );
              
              return _AnimatedLikeButton(
                isLiked: currentPost.isLiked,
                likeCount: currentPost.likeCount,
                onTap: () {
                  context.read<PostInteractionBloc>().add(
                    PostInteractionEvent.likePost(postId: post.id),
                  );
                },
              );
            },
          ),
          
          const SizedBox(width: 16),
          
          // Comment Button
          _ActionButton(
            icon: Icons.mode_comment_outline,
            label: '${post.commentCount}',
            onTap: () => _showComments(context),
          ),
          
          const SizedBox(width: 16),
          
          // Share Button
          _ActionButton(
            icon: Icons.share,
            label: 'Share',
            onTap: () => _sharePost(context),
          ),
        ],
      ),
    );
  }
}

class _AnimatedLikeButton extends StatefulWidget {
  final bool isLiked;
  final int likeCount;
  final VoidCallback onTap;

  const _AnimatedLikeButton({
    required this.isLiked,
    required this.likeCount,
    required this.onTap,
  });

  @override
  State<_AnimatedLikeButton> createState() => _AnimatedLikeButtonState();
}

class _AnimatedLikeButtonState extends State<_AnimatedLikeButton> 
    with SingleTickerProviderStateMixin {
  late AnimationController _controller;
  late Animation<double> _scaleAnimation;

  @override
  void initState() {
    super.initState();
    _controller = AnimationController(
      vsync: this,
      duration: const Duration(milliseconds: 200),
    );
    
    _scaleAnimation = Tween<double>(begin: 1.0, end: 1.3).animate(
      CurvedAnimation(parent: _controller, curve: Curves.easeOut),
    );
  }

  @override
  void didUpdateWidget(covariant _AnimatedLikeButton oldWidget) {
    super.didUpdateWidget(oldWidget);
    // Trigger animation only when like status changes to TRUE
    if (widget.isLiked && !oldWidget.isLiked) {
      _controller.forward().then((_) => _controller.reverse());
    }
  }

  @override
  void dispose() {
    _controller.dispose();
    super.dispose();
  }

  @override
  Widget build(BuildContext context) {
    return ScaleTransition(
      scale: _scaleAnimation,
      child: _ActionButton(
        icon: widget.isLiked ? Icons.favorite : Icons.favorite_border,
        label: '${widget.likeCount}',
        color: widget.isLiked ? Colors.red : Colors.grey,
        onTap: widget.onTap,
      ),
    );
  }
}
```

**Explanation:**

- **didUpdateWidget**: Detects when the parent BLoC rebuilds the widget with new data. Triggers the "pop" animation when `isLiked` changes to true.
- **ScaleTransition**: Makes the heart icon grow briefly then shrink back, providing satisfying feedback.
- **BlocBuilder**: Listens to the `PostInteractionBloc`, ensuring the specific post's state is always current.

---

## **45.6 Threaded Comments System**

Handling nested comment structures is a common complexity in social apps.

### **Domain Entity for Threads**

```dart
// lib/features/feed/domain/entities/comment.dart
@freezed
class Comment with _$Comment {
  const factory Comment({
    required String id,
    required String postId,
    required User author,
    required String content,
    required DateTime createdAt,
    required List<Comment> replies, // Recursive structure
    required int replyCount,
  }) = _Comment;
}
```

### **Recursive UI for Threads**

```dart
// lib/features/feed/presentation/widgets/comment_tile.dart
class CommentTile extends StatelessWidget {
  final Comment comment;
  final int depth; // Track nesting level

  const CommentTile({
    super.key,
    required this.comment,
    this.depth = 0,
  });

  @override
  Widget build(BuildContext context) {
    return Column(
      crossAxisAlignment: CrossAxisAlignment.start,
      children: [
        // Main comment content
        Padding(
          padding: EdgeInsets.only(left: depth * 16.0), // Indent replies
          child: Row(
            crossAxisAlignment: CrossAxisAlignment.start,
            children: [
              CircleAvatar(
                backgroundImage: NetworkImage(comment.author.avatarUrl),
                radius: 16,
              ),
              const SizedBox(width: 8),
              Expanded(
                child: Column(
                  crossAxisAlignment: CrossAxisAlignment.start,
                  children: [
                    Text(
                      comment.author.name,
                      style: const TextStyle(fontWeight: FontWeight.bold),
                    ),
                    Text(comment.content),
                    Row(
                      children: [
                        Text(_formatTimestamp(comment.createdAt)),
                        TextButton(
                          onPressed: () => _replyToComment(context),
                          child: const Text('Reply'),
                        ),
                      ],
                    ),
                  ],
                ),
              ),
            ],
          ),
        ),
        
        // Recursively render replies
        if (comment.replies.isNotEmpty)
          ...comment.replies.map(
            (reply) => CommentTile(comment: reply, depth: depth + 1),
          ),
        
        // Show "View more replies" if truncated
        if (comment.replyCount > comment.replies.length)
          TextButton(
            onPressed: () => _loadMoreReplies(context),
            child: Text('View ${comment.replyCount - comment.replies.length} more replies'),
          ),
      ],
    );
  }
  
  void _replyToComment(BuildContext context) { /* ... */ }
  void _loadMoreReplies(BuildContext context) { /* ... */ }
}
```

**Explanation:**

- **Recursive Widget**: `CommentTile` calls itself for `replies`. This naturally handles infinite nesting.
- **Depth Tracking**: `depth` parameter adds padding (`left: depth * 16.0`) to visually distinguish reply levels.
- **Truncation**: APIs usually don't return all replies at once. The "View more replies" button triggers a pagination event for that specific comment thread.

---

## **Chapter Summary**

In this chapter, we built a high-performance Social Media Feed:

### **Key Takeaways:**

1. **Cursor-Based Pagination**: Essential for dynamic feeds. Cursors prevent duplicate items when new posts are inserted at the top while the user scrolls.

2. **Dual Data Source Strategy**: REST for initial history (reliable), WebSocket for live updates (fast). Merge these streams in the Presentation layer.

3. **Optimistic UI**: Update local state immediately for likes/comments to maintain "60fps" feel. Roll back if API fails. Conflict resolution corrects counts if server differs.

4. **Media Efficiency**: `cached_network_image` with `memCacheWidth` is critical to prevent OOM crashes in infinite lists. Video controllers must be strictly managed and disposed.

5. **Recursive UI**: Handling nested comments with recursive widgets (`CommentTile` calling itself) is cleaner than flattening the data.

6. **Separation of Concerns**: Split `FeedBloc` (loading/pagination) from `PostInteractionBloc` (likes/comments). A failed API call for a like shouldn't reload the entire feed.

### **Performance Checklist:**
- ✅ Images resized to appropriate resolution before caching
- ✅ Video players disposed immediately when off-screen
- ✅ ListView uses `itemBuilder` (lazy loading), not `children: []`
- ✅ BLoC `buildWhen` prevents unnecessary widget rebuilds
- ✅ WebSocket reconnection logic prevents connection zombies

---

## **Next Steps**

In the next chapter, **Chapter 46: Project 3 - Productivity/Task Management App**, we will build an offline-first application. You'll learn how to implement a robust offline architecture, synchronize local database with server, schedule background tasks for reminders, and integrate with device calendars.

---

**End of Chapter 45**